# Преобразование файлов в формат $TeX$
На предыдущем этапе (*см. arXiv-API.ipynb*) посредством arXiv API были скачаны на диск исходники статей. Данные файлы не имееют расширения. Какие-то файлы заархивированы (*tar* или *gzip*), какие-то -- нет (*pdf* или *tex*). Если файл не является архивом, то просматривается его содержание: если файл содержит подстроку "%!PS-Adobe", то он отмечается как *pdf* (либо когда ошибка чтения файла), иначе -- как *tex*.   
  
[Здесь]() будет ссылка на архив с исходниками. А [здесь](https://drive.google.com/open?id=0B_NefGKQ1pCsQjhLSTZ5Mjlqbms) ссылка на метаданные.

Загрузка необходимых библиотек

In [1]:
import os
import gzip
import tarfile
import time
import pandas as pd

Указание пути к папке с исходниками (*path_to_sources*) и к папке, куда будут записываться преобразованные файлы (*path_to_tex*)

In [4]:
path_to_sources = 'C:/Users/user/Downloads/arxiv/source/math.PR/'
path_to_tex = 'C:/Users/user/Downloads/arxiv/source/valid/'

# Метаданные
path_meta = 'D:/_data_science/@@projects/2017_diploma-nlp/data/'
queries = pd.read_csv(path_meta + "unique_queries.csv", encoding='cp1251', index_col='id')

Реализация функции, которая преобразовывает файл в формат *tex* (*pdf*) и сохраняет в указанную папку.

Если в архиве содержится несколько *tex*-файлов, то они объединяются в один документ.

In [12]:
def file_to_tex(filename, path_to_sources=path_to_sources, path_to_tex=path_to_tex):
    '''
    Description
    '''
    # определяем главную категорию
    # если нет одноименной папки в path_to_tex, то создаем
    #
    category = queries.loc[filename].primary_category
    if not os.path.exists(path_to_tex + category):
        os.makedirs(path_to_tex + category)
    
    try:
        tar = tarfile.open(path_to_sources + filename)
        names = tar.getnames()   
        names_tex = [name for name in names if '.tex' in name]
        if len(names_tex) > 1:                                      # если в архиве несколько tex-файлов
            s = b''
            for name in names_tex:
                part = tar.extractfile(name).read()
                s += part
        elif len(names_tex) == 1:
            s = tar.extractfile(names_tex[0]).read()
        tar.close() 
                
        f = open(path_to_tex + category + '/' + filename + '.tex', 'wb')
        f.write(s)
        f.close()
                
    except tarfile.ReadError:
        fread = open(path_to_sources + filename,'rb').read()
            
        try:
            s = gzip.decompress(fread)
            fwrite = open(path_to_tex + category + '/' + filename + '.tex','wb')
            fwrite.write(s)
            fwrite.close()
        except OSError:
            
            try:
                f = open(path_to_sources + filename).read(10)
                if f == '%!PS-Adobe':
                    os.rename(path_to_sources + filename, path_to_tex + category + '/' + filename + '.pdf') # pdf
                else:
                    os.rename(path_to_sources + filename, path_to_tex + category + '/' + filename + '.tex')
            except Exception:
                os.rename(path_to_sources + filename, path_to_tex + category + '/' + filename + '.pdf') # maybe pdf

Выполним процедуру преобразования для имеющихся файлов

In [6]:
inds = queries.index
source_files = os.listdir(path_to_sources)
tex_files = os.listdir(path_to_tex)

In [13]:
%%time

counter = 0
for file in source_files:
    if file not in tex_files:
        counter += 1
        try:
            file_to_tex(file)
        except Exception:
            continue
print('%d files was processed!' % counter)

18905 files was processed!
Wall time: 25min 57s


>**Замечание!**  
Скорость преобразования, как можно заметить, **~3 файл/c**. Значит, для обработки ~160K файлов потребуется **~15 часов**. Эта оценка сильно зависит от того как много среди этих файлов архивов и от мощности компьютера.  

[Здесь]() будет ссылка на архив c полученными tex-файлами.